<u><h1>III- Demo. Data Evaluation </h1></u>

An example of SSH reconstruction has been produced in the "baseline_oi.ipynb" notebook. Here, a data evaluation is proposed. 

The notebook is structured as follow:

     1) reading of reference and reconstructed SSH fields, 
     2) make field on similar spatio-temporal grid and 
     3) comparison of reconstrusted and reference SSH fields (statistical/spectral comparison).

- the statistical comparison is based on the RMSE-based score $RMSE_{S}$ defined as:

$$RMSE_{S}(t) = 1 - \frac{RMSE(t)}{RMS(SSH_{true})}$$


where RMS is the root mean square function, and with:


$$RMSE(t) = \sqrt{ \frac{1}{N} \sum_{i=1}^N (SSH_{reconstruction}(t,i) - SSH_{true}(t,i))^2   }$$


N is the number of pixels included in the study domain.

- the spectral analysis is based on the wavenumber-frequency power spectral density score $PSD_{S}^{wf}$ defined as:

$$PSD_{S}^{wf} = 1 - \frac{PSD^{wf}(SSH_{reconstruction} - SSH_{true})}{PSD^{wf}(SSH_{true})}$$

The **Leaderboard** summarizes the following key metrics:


In [ ]:
import xarray as xr
import numpy
import hvplot.xarray
import pyinterp
import dask
import warnings
import xrft
import logging
import os
import sys
warnings.filterwarnings('ignore')

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [ ]:
cluster = dask.distributed.LocalCluster()
client = dask.distributed.Client(cluster)
client

In [ ]:
sys.path.append('..')

In [ ]:
from src.mod_oi import *
from src.mod_inout import *
from src.mod_regrid import *
from src.mod_eval import *
from src.mod_plot import *

In [ ]:
time_min = numpy.datetime64('2012-10-22')                # domain min time
time_max = numpy.datetime64('2012-12-02')                # domain max time

In [ ]:
input_file = '../results/ssh_reconstruction_2012-10-22-2012-12-02_jason1.nc'

In [ ]:
ds_oi1_grid = xr.open_dataset(input_file)

In [ ]:
# Note: dc_ref is used for reggriding step
dc_ref = xr.open_mfdataset('../dc_ref/*.nc', combine='nested', concat_dim='time', parallel=True)
dc_ref

In [ ]:
# Note: dc_ref_sample is used for reggriding step (daily mean is enougth !!!!!)
dc_ref_sample = dc_ref.sel(time=slice(time_min, time_max)).resample(time='1D').mean()
del dc_ref
dc_ref_sample

In [ ]:
# Regrid    
ds_oi1_regrid = oi_regrid(ds_oi1_grid, dc_ref_sample)

In [ ]:
# Eval
rmse_t_oi1, rmse_xy_oi1 = rmse_based_scores(ds_oi1_regrid, dc_ref_sample)
psd_oi1 = psd_based_scores(ds_oi1_regrid, dc_ref_sample)